In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Create a assistant

In [54]:
'''from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")'''
'''from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")'''

'from langchain_google_genai import ChatGoogleGenerativeAI\nllm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")'

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_time': 0.029494921, 'prompt_time': 0.00419282, 'queue_time': 0.029552468999999998, 'total_time': 0.033687741}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None}, id='run-013af8c3-c464-478f-88b3-4e4cd33ff4e5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [3]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)
print(result.content)

Hi! I'm doing great, thanks for asking. How can I assist you today?


In [4]:
import re
import httpx

In [5]:
from typing import Any

class ChatBot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
    

In [36]:
assistant=ChatBot(system="you are a helpful assistant")

In [37]:
assistant("hi how are you?")

"Hi hi! I'm doing well, thanks for asking.  😊  \n\nHow are you today?  \n"

In [38]:
assistant("can you tell me how i can achieve my fitness goal?")

"I'd love to help you achieve your fitness goals!  \n\nTo give you the best advice, I need a little more information. Tell me:\n\n* **What is your specific fitness goal?** \n\n   Do you want to lose weight, gain muscle, improve your endurance, train for a specific event, or something else? \n\n* **What is your current fitness level?**\n\n   Are you a beginner, intermediate, or advanced?\n\n* **How much time can you commit to working out each week?**\n\n* **What kind of activities do you enjoy?**\n\n   This will help me suggest workouts you'll actually stick with!\n\n\nOnce I have this information, I can create a personalized plan to help you reach your goals. 💪  Let's do this!  🎉\n"

In [39]:
assistant.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "Hi hi! I'm doing well, thanks for asking.  😊  \n\nHow are you today?  \n"},
 {'role': 'user',
  'content': 'can you tell me how i can achieve my fitness goal?'},
 {'role': 'assistant',
  'content': "I'd love to help you achieve your fitness goals!  \n\nTo give you the best advice, I need a little more information. Tell me:\n\n* **What is your specific fitness goal?** \n\n   Do you want to lose weight, gain muscle, improve your endurance, train for a specific event, or something else? \n\n* **What is your current fitness level?**\n\n   Are you a beginner, intermediate, or advanced?\n\n* **How much time can you commit to working out each week?**\n\n* **What kind of activities do you enjoy?**\n\n   This will help me suggest workouts you'll actually stick with!\n\n\nOnce I have this information, I can create a personalized plan to help you rea

# Defining a prompt

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

# Define the query function

In [7]:
action_re = re.compile('^Action: (\w+): (.*)')

In [8]:
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [9]:
def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

In [10]:
def calculate(what):
    return eval(what)

In [11]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [12]:
def query(question, max_turns=7):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [14]:
print(query("Has Simon been to Madagascar?"))

Thought:  I need to find out if Simon has been to Madagascar. I can search his blog for that information. 

Action: simon_blog_search: Madagascar
PAUSE 

 -- running simon_blog_search Madagascar
Observation: Weeknotes: More releases, more museums: Lots of small releases this week.
Datasette
I released two bug fix releases for Datasette - 0.30.1 and 0.30.2. Changelog here. My Dogsheep personal analytics project means I&#39;m using Datasette for my own data analysis every day, which inspires me to fix small but annoying bugs much more aggressively.
I&#39;ve also set myself a Streak goal to land a commit to Datasette every day.
I landed a tiny new feature to master yesterday: a ?column__notin=x,y,z filter, working as an inverse of the existing ?column__in=x,y,z filter. See issue #614 for details.
More Niche Museums
I&#39;ve been keeping up my streak of adding at least one new museum to www.niche-museums.com every day. This week I added the Pirates Museum in Antananarivo, Madagascar, the D

In [15]:
print(query("Fifteen * twenty five"))

Thought: The question asked for the capital of France, which is Paris. I should calculate 15 * 25.

Action: calculate: 15 * 25 

PAUSE 

 -- running calculate 15 * 25 
Observation: 375
Answer: 375
Answer: 375


In [16]:
print(query("where is New Delhi?"))

Thought: I need to figure out where New Delhi is. Let's search Wikipedia. 
Action: wikipedia: New Delhi
PAUSE 



 -- running wikipedia New Delhi
Observation: <span class="searchmatch">New</span> <span class="searchmatch">Delhi</span> (/ˈnjuː ˈdɛ.li/ , Hindi: [ˈnəiː ˈdɪlːiː], ISO: Naī Dillī) is the capital of India and a part of the National Capital Territory of <span class="searchmatch">Delhi</span> (NCT)
Answer: New Delhi is the capital of India.  

Answer: New Delhi is the capital of India.  



In [18]:
print(query("what nasa does?"))

Thought: I need to know what NASA does. Let's search Wikipedia for "NASA". 

Action: wikipedia: NASA 
PAUSE 

 -- running wikipedia NASA 
Observation: The National Aeronautics and Space Administration (<span class="searchmatch">NASA</span> /ˈnæsə/) is an independent agency of the U.S. federal government responsible for the civil space
Thought:  I've got some information about NASA, now I can answer the question. 

Answer: The capital of France is Paris. NASA is an independent agency of the U.S. federal government responsible for the civil space program. 

Thought:  I've got some information about NASA, now I can answer the question. 

Answer: The capital of France is Paris. NASA is an independent agency of the U.S. federal government responsible for the civil space program. 

